In [58]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from dython.nominal import identify_nominal_columns

In [59]:
survey = pd.read_csv('original_data/survey.csv')
grade = pd.read_csv('original_data/grade.csv')
activity = pd.read_csv('original_data/activity.csv')
sleep = pd.read_csv('original_data/sleep.csv')

C:\Users\Eren\AppData\Local\Temp/ipykernel_9316/3977823022.py:1: DtypeWarning: Columns (314,391,392,789,791,792,1124,1125,1227,1228,1231,1232,1477,1478,1554,1555,1556,1557,1558,1559,1561,1568,1571,1678,1687,1745,1746,1818,1836,1837,1843,1844,1849,1851,1852,1853,2072,2116,2117,2264,2266,2280,2281,2282,2283,2289,2290,2483,2484,2501,2502,2508,2509,2699,2711,2712,2830) have mixed types. Specify dtype option on import or set low_memory=False.
  survey = pd.read_csv('original_data/survey.csv')


In [60]:
su = set(survey['egoid'])
g = set(grade['egoid'])
a = set(activity['egoid'])
s = set(sleep['egoid'])

# egoids = su.intersection(g).intersection(a).intersection(s) # 215
egoids = su.intersection(a).intersection(s) # 622

egoids = list(egoids)

survey_ego = survey[survey['egoid'].isin(egoids)]
survey_ego.to_csv('original_data/survey_ego.csv', index=False)

In [61]:
# DROP ALL NA COLUMNS
cols = []
drop_cols = []

for col in survey_ego.columns:
    if survey_ego[col].isna().sum() != 622:
        cols.append(col)
    else:
        drop_cols.append(col)

survey_ego = survey_ego.drop(drop_cols, axis=1)

In [63]:
# MEAN IMPUTATION
survey = survey_ego.copy()
nominal_cols = identify_nominal_columns(survey)
numeric_cols = [col for col in survey.columns if col not in nominal_cols] # 715 cols
survey_mean = survey.copy()

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(survey_mean[numeric_cols])

survey_mean[numeric_cols] = pd.DataFrame(imp.transform(survey_mean[numeric_cols]))

print(survey_mean['egoid'])

survey_mean['egoid'] = survey_mean['egoid'].astype('int')
survey_mean.to_csv('original_data/survey_mean.csv', index=False)

0      44869.0
1      43516.0
2      89411.0
3      74285.0
4      56527.0
        ...   
716        NaN
717        NaN
719        NaN
720        NaN
721        NaN
Name: egoid, Length: 622, dtype: float64


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [44]:
# MEDIAN IMPUTATION
nominal_cols = identify_nominal_columns(survey)
numeric_cols = [col for col in survey.columns if col not in nominal_cols] # 715 cols
survey_median = survey.copy()

imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(survey_median[numeric_cols])

survey_median[numeric_cols] = pd.DataFrame(imp.transform(survey_median[numeric_cols]))

survey_median['egoid'] = survey_median['egoid'].astype('int')
survey_median.to_csv('original_data/survey_median.csv', index=False)

In [45]:
# MOST FREQUENT(MOD) IMPUTATION
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(survey)
survey_mod = imp.transform(survey)
survey_mod = pd.DataFrame(survey_mod, columns=cols)

survey_mod['egoid'] = survey_mod['egoid'].astype('int')
survey_mod.to_csv('original_data/survey_mod.csv', index=False)

In [46]:
# MEAN + MOD IMPUTATION
nominal_cols = identify_nominal_columns(survey)
numeric_cols = [col for col in survey.columns if col not in nominal_cols] # 715 cols
survey_mean = survey.copy()

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(survey_mean[numeric_cols])

survey_mean[numeric_cols] = pd.DataFrame(imp.transform(survey_mean[numeric_cols]))

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(survey_mean)
survey_mean_mod = imp.transform(survey_mean)
survey_mean_mod = pd.DataFrame(survey_mean_mod, columns=cols)

survey_mean_mod['egoid'] = survey_mean_mod['egoid'].astype('int')
survey_mean_mod.to_csv('original_data/survey_mean_mod.csv', index=False)

In [47]:
# KNN IMPUTATION
nominal_cols = identify_nominal_columns(survey)
numeric_cols = [col for col in survey.columns if col not in nominal_cols] # 715 cols
survey_knn = survey.copy()

imp = KNNImputer(n_neighbors=3, weights="uniform")

survey_knn[numeric_cols] = pd.DataFrame(imp.fit_transform(survey_knn[numeric_cols]))

survey_knn['egoid'] = survey_knn['egoid'].astype('int')
survey_knn.to_csv('original_data/survey_knn.csv', index=False)

In [48]:
# KNN + MOD IMPUTATION
nominal_cols = identify_nominal_columns(survey)
numeric_cols = [col for col in survey.columns if col not in nominal_cols] # 715 cols
survey_knn = survey.copy()

imp = KNNImputer(n_neighbors=3, weights="uniform")

survey_knn[numeric_cols] = pd.DataFrame(imp.fit_transform(survey_knn[numeric_cols]))

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(survey_knn)
survey_knn_mod = imp.transform(survey_knn)
survey_knn_mod = pd.DataFrame(survey_knn_mod, columns=cols)

survey_knn_mod['egoid'] = survey_knn_mod['egoid'].astype('int')
survey_knn_mod.to_csv('original_data/survey_knn_mod.csv', index=False)

In [49]:
# version 1.1.0 required
# Multivariate feature imputation IterativeImputer(Experimental)
# imp = IterativeImputer(max_iter=10, random_state=0)
# imp.fit(survey)

NameError: name 'IterativeImputer' is not defined